In [ ]:
import cv2
import numpy as np
import math
from google.colab.patches import cv2_imshow


def NCC(img1, img2):
  img1 = img1 - img1.mean(axis = 0)
  img2 = img2 - img2.mean(axis = 0)
  normImg1 = img1 / np.linalg.norm(img1)
  normImg2 = img2 / np.linalg.norm(img2)
  return np.sum(normImg1 * normImg2)

def alignment(blue, other):
  
  min = -1
  for i in range(-15, 16):
    for j in range(-15, 16):
      ncc = NCC(blue, np.roll(other, [i, j], axis = (0, 1)))
      if(ncc > min):
        min = ncc
        rollAmount = [i, j]

  return rollAmount

def SSD(i1, i2):
  x = np.subtract(i1, i2)
  ssd = np.multiply(x, x)
  return np.sum(ssd)

img = cv2.imread('/content/sample_data/00907v.jpg', -1) # path of image

img2 = cv2.equalizeHist(img) # Histogram Equalization

#cropped_image = img[9:1014, 8:388] # Without Histogram Equalization
cropped_image = img2[9:1014, 8:388] # With Histogram Equalization

height = 335
blue = cropped_image[: height]
green = cropped_image[height : 2 * height]
red = cropped_image[2 * height : 3 * height]

greenToBlue = alignment(blue, green)
redToBlue = alignment(blue, red)

print(greenToBlue) # roll Amount of Green
print(redToBlue) # roll Amount of Red

newGreen = np.roll(green, greenToBlue, axis = (0, 1))
newRed = np.roll(red, redToBlue, axis = (0, 1))

noAlign = cv2.merge([blue, green, red]) # no alignment operation made, just merge 3 channels

result = cv2.merge([blue, newGreen, newRed]) # normal result with histogram equalization

gammaResult = np.array(255*(result/255)** 1.5, dtype='uint8') # gamma correction result

gaussian = cv2.GaussianBlur(gammaResult, (3,3), 0) # gaussian blured over gamma correction
laplacian = cv2.Laplacian(gaussian,cv2.CV_64F) # laplacian filter over gaussian blured

finalResult = (np.subtract(gaussian, laplacian)) # final result

# to show in google colab, it supports only this form
cv2_imshow(result)
cv2_imshow(gammaResult)
cv2_imshow(finalResult)
# to get .jpg outputs
cv2.imwrite('31421v - non aligned result.jpg', noAlign)
cv2.imwrite('31421v - normal result.jpg', result)
cv2.imwrite('31421v - gamma result.jpg', gammaResult)
cv2.imwrite('31421v - final result.jpg', finalResult)